In [1]:
import json
import os
from collections import Counter

import gensim
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/lyu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
sentences_cnn = []
counts_general = dict()
counts_cnn = dict()
counts_nypost = dict()
counts_general[0] = 0
counts_cnn[0] = 0
counts_nypost[0] = 0
for fname in os.listdir('./data/nypost/'):
    if len(fname) > 12:
        print(fname)
        df = pd.read_csv(os.path.join('./data/nypost', fname))
        for idx, row in df.iterrows():
            for st in sent_tokenize(str(row['text'])):
                sts_list = gensim.utils.simple_preprocess(st)
                hist = Counter(sts_list)
                for k in hist:
                    counts_general[0] += 1
                    counts_nypost[0] += 1
                    counts_general[k] = hist[k] + counts_general.get(k, 0)
                    counts_nypost[k] = hist[k] + counts_general.get(k, 0)
                sentences_cnn.append(sts_list)

sentences_nypost = []
for fname in os.listdir('./data/CNN/'):
    if len(fname) > 12:
        print(fname)
        df = pd.read_csv(os.path.join('./data/CNN', fname))
        for idx, row in df.iterrows():
            for st in sent_tokenize(str(row['text'])):
                sts_list = gensim.utils.simple_preprocess(st)
                hist = Counter(sts_list)
                for k in hist:
                    counts_general[0] += 1
                    counts_cnn[0] += 1
                    counts_cnn[k] = hist[k] + counts_cnn.get(k, 0)
                    counts_general[k] = hist[k] + counts_general.get(k, 0)
                sentences_nypost.append(sts_list)

nypost_gun_page1to10000_timeto.csv


KeyboardInterrupt: 

In [ ]:
counts = dict()
counts[0] = 0
for sts in sentences_nypost + sentences_cnn:
    hist = Counter(sts)
    for k in hist:
        counts[0] += 1
        counts[k] = hist[k] + counts.get(k, 0)
json.dump(counts, open(os.path.join('./models', 'general_word_freq_add.json'), 'w'))

In [ ]:
json.dump(counts_general, open(os.path.join('./models', 'general_word_freq_add.json'), 'w'))
json.dump(counts_nypost, open(os.path.join('./models', 'nypost_word_freq_add.json'), 'w'))
json.dump(counts_cnn, open(os.path.join('./models', 'cnn_word_freq_add.json'), 'w'))

In [ ]:
opts = {'dims': 100,
        'window': 5,
        'n_cpu': -1,
        'min_count': 30,
        'vocab_size': 10000,
        'sample': 0.0001,
        'n_iter': 30
        }
model = gensim.models.Word2Vec(sentences=sentences_cnn + sentences_nypost,
                               vector_size=opts['dims'],
                               window=opts['window'],
                               workers=opts['n_cpu'],
                               sg=1,
                               hs=0,
                               negative=5,
                               min_count=opts['min_count'],
                               max_final_vocab=opts['vocab_size'],
                               sample=opts['sample'],
                               epochs=opts['n_iter']
                               )
model.save(os.path.join('./models', 'general_add.model'))

In [ ]:
opts = {'dims': 100,
        'window': 5,
        'n_cpu': -1,
        'min_count': 30,
        'vocab_size': 10000,
        'sample': 0.0001,
        'n_iter': 30
        }
model = gensim.models.Word2Vec(sentences=sentences_nypost,
                               vector_size=opts['dims'],
                               window=opts['window'],
                               workers=opts['n_cpu'],
                               sg=1,
                               hs=0,
                               negative=5,
                               min_count=opts['min_count'],
                               max_final_vocab=opts['vocab_size'],
                               sample=opts['sample'],
                               epochs=opts['n_iter']
                               )

model.save(os.path.join('./models', 'nypost_add.model'))

In [ ]:
opts = {'dims': 100,
        'window': 5,
        'n_cpu': -1,
        'min_count': 30,
        'vocab_size': 10000,
        'sample': 0.0001,
        'n_iter': 30
        }
model = gensim.models.Word2Vec(sentences=sentences_cnn,
                               vector_size=opts['dims'],
                               window=opts['window'],
                               workers=opts['n_cpu'],
                               sg=1,
                               hs=0,
                               negative=5,
                               min_count=opts['min_count'],
                               max_final_vocab=opts['vocab_size'],
                               sample=opts['sample'],
                               epochs=opts['n_iter']
                               )

model.save(os.path.join('./data', 'models', 'cnn_add.model'))

In [ ]:
import json
import os
import pickle
from abc import ABC

import gensim
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.cross_decomposition import CCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
stopWords_nltk = set(stopwords.words('english'))
stopWords_spc = {'those', 'on', 'own', '’ve', 'yourselves', 'around', 'between', 'four', 'been', 'alone', 'off', 'am',
                 'then', 'other', 'can', 'regarding', 'hereafter', 'front', 'too', 'used', 'wherein', '‘ll', 'doing',
                 'everything', 'up', 'onto', 'never', 'either', 'how', 'before', 'anyway', 'since', 'through', 'amount',
                 'now', 'he', 'was', 'have', 'into', 'because', 'not', 'therefore', 'they', 'n’t', 'even', 'whom', 'it',
                 'see', 'somewhere', 'thereupon', 'nothing', 'whereas', 'much', 'whenever', 'seem', 'until', 'whereby',
                 'at', 'also', 'some', 'last', 'than', 'get', 'already', 'our', 'once', 'will', 'noone', "'m", 'that',
                 'what', 'thus', 'no', 'myself', 'out', 'next', 'whatever', 'although', 'though', 'which', 'would',
                 'therein', 'nor', 'somehow', 'whereupon', 'besides', 'whoever', 'ourselves', 'few', 'did', 'without',
                 'third', 'anything', 'twelve', 'against', 'while', 'twenty', 'if', 'however', 'herself', 'when', 'may',
                 'ours', 'six', 'done', 'seems', 'else', 'call', 'perhaps', 'had', 'nevertheless', 'where', 'otherwise',
                 'still', 'within', 'its', 'for', 'together', 'elsewhere', 'throughout', 'of', 'others', 'show', '’s',
                 'anywhere', 'anyhow', 'as', 'are', 'the', 'hence', 'something', 'hereby', 'nowhere', 'latterly', 'say',
                 'does', 'neither', 'his', 'go', 'forty', 'put', 'their', 'by', 'namely', 'could', 'five', 'unless',
                 'itself', 'is', 'nine', 'whereafter', 'down', 'bottom', 'thereby', 'such', 'both', 'she', 'become',
                 'whole', 'who', 'yourself', 'every', 'thru', 'except', 'very', 'several', 'among', 'being', 'be',
                 'mine', 'further', 'n‘t', 'here', 'during', 'why', 'with', 'just', "'s", 'becomes', '’ll', 'about',
                 'a', 'using', 'seeming', "'d", "'ll", "'re", 'due', 'wherever', 'beforehand', 'fifty', 'becoming',
                 'might', 'amongst', 'my', 'empty', 'thence', 'thereafter', 'almost', 'least', 'someone', 'often',
                 'from', 'keep', 'him', 'or', '‘m', 'top', 'her', 'nobody', 'sometime', 'across', '‘s', '’re',
                 'hundred', 'only', 'via', 'name', 'eight', 'three', 'back', 'to', 'all', 'became', 'move', 'me', 'we',
                 'formerly', 'so', 'i', 'whence', 'under', 'always', 'himself', 'in', 'herein', 'more', 'after',
                 'themselves', 'you', 'above', 'sixty', 'them', 'your', 'made', 'indeed', 'most', 'everywhere',
                 'fifteen', 'but', 'must', 'along', 'beside', 'hers', 'side', 'former', 'anyone', 'full', 'has',
                 'yours', 'whose', 'behind', 'please', 'ten', 'seemed', 'sometimes', 'should', 'over', 'take', 'each',
                 'same', 'rather', 'really', 'latter', 'and', 'ca', 'hereupon', 'part', 'per', 'eleven', 'ever', '‘re',
                 'enough', "n't", 'again', '‘d', 'us', 'yet', 'moreover', 'mostly', 'one', 'meanwhile', 'whither',
                 'there', 'toward', '’m', "'ve", '’d', 'give', 'do', 'an', 'quite', 'these', 'everyone', 'towards',
                 'this', 'cannot', 'afterwards', 'beyond', 'make', 'were', 'whether', 'well', 'another', 'below',
                 'first', 'upon', 'any', 'none', 'many', 'serious', 'various', 're', 'two', 'less', '‘ve'}
stopWords_s = stopWords_spc | stopWords_nltk | STOPWORDS

In [ ]:
model_general = gensim.models.Word2Vec.load(os.path.join('./models', 'general_add.model'))

vocab = list(sorted(list(model_general.wv.index_to_key)))
vocab = [w for w in vocab if w not in stopWords_s]
mtx = np.vstack([model_general.wv[w] for w in vocab])

clustering = KMeans(n_clusters=300).fit(mtx)
res = {}
t_align = {}
for c, w in zip(clustering.labels_, vocab):
    t_align[w] = c
    c = str(c)
    if c not in res:
        res[c] = []
    res[c].append(w)

In [ ]:
json.dump(res, open(os.path.join('./models', 'news_clustering_add.tpc'), 'w'))

t_num = []
t_words = []
t_name = []
for k in res.keys():
    t_num.append(k)
    t_words.append(res[k])
    t_name.append([])
pd.DataFrame({'id': t_num, 'name': t_name, 'words': t_words}).to_csv('./news_topics_add.csv')

In [ ]:
model_general=gensim.models.Word2Vec.load(os.path.join('./models', 'general_add.model'))
model_nypost = gensim.models.Word2Vec.load(os.path.join('./models', 'nypost_add.model'))
model_cnn = gensim.models.Word2Vec.load(os.path.join('./models', 'cnn_add.model'))

In [ ]:
shared_vocab = set(model_nypost.wv.key_to_index).intersection(set(model_cnn.wv.key_to_index)).intersection(set(model_general.wv.key_to_index))
stopWords = list(stopWords_s & shared_vocab)

In [ ]:
class Aligner(ABC):
    def __init__(self, method, source, target, w2id, id2w, mtxA, mtxB, trainvoc):
        self.method = method
        self.src = source
        self.tgt = target
        self.w2idA = w2id
        self.id2wB = id2w
        self.mtxA = mtxA
        self.mtxB = mtxB
        self.anchors = trainvoc

    def translate_mtx(self, mtx):
        """
        MTX -> MTX
        """
        pass

    def encode_input(self, words):
        """
        [STRING] -> MTX
        """
        embs = [self.mtxA[self.w2idA[w], :] for w in words]
        return np.vstack(embs)

    def decode_output(self, mtx, k=1):
        """
        MTX -> [[STRING]]
        """
        similarities = cosine_similarity(mtx, self.mtxB)
        most_similar = np.argsort(similarities, axis=1)[:, ::-1]
        topsims = np.sort(similarities, axis=1)[:, ::-1][:, :k]
        res = [[self.id2wB[i] for i in row[:k]] for row in most_similar]
        return res, topsims

    def translate_word(self, word, k=1):
        """
        STRING -> STRING
        """
        encoding = self.encode_input([word])
        translated = self.translate_mtx(encoding)
        decoded = self.decode_output(translated, k=k)
        return decoded[0][:k]

    def translate_words(self, words, k=1):
        """
        [STRING] -> [STRING]
        """
        encoding = self.encode_input(words)
        translated = self.translate_mtx(encoding)
        decoded, simscores = self.decode_output(translated, k=k)
        return decoded, simscores


class CCAAligner(Aligner):
    def set_params(self, cca):
        self.cca = cca

    def translate_mtx(self, mtx):
        return mtx

    def translate_word(self, word, k=1):
        tmpA = self.mtxA
        tmpB = self.mtxB
        self.mtxA, self.mtxB = self.cca.transform(tmpA, tmpB)
        res = super().translate_word(word, k=k)
        self.mtxA = tmpA
        self.mtxB = tmpB
        return res

    def translate_words(self, words, k=1):
        tmpA = self.mtxA
        tmpB = self.mtxB
        self.mtxA, self.mtxB = self.cca.transform(tmpA, tmpB)
        res, simscores = super().translate_words(words, k=k)
        self.mtxA = tmpA
        self.mtxB = tmpB
        return res, simscores


class SVDAligner(Aligner):
    def set_params(self, T):
        self.T = T

    def translate_mtx(self, mtx):
        return mtx.dot(self.T)

In [ ]:
def align_cca(source, target):
    N_dims = source.shape[1]
    cca = CCA(n_components=N_dims, max_iter=2000)
    cca.fit(source, target)
    return cca


def align_svd(source, target):
    product = np.matmul(source.transpose(), target)
    U, s, V = np.linalg.svd(product)
    T = np.matmul(U, V)
    return T


def get_cca_aligner(model_a, model_b, anchorlist):
    # get wordmaps
    awords = list(sorted(list(model_a.wv.key_to_index)))
    bwords = list(sorted(list(model_b.wv.key_to_index)))
    w2idA = {w: i for i, w in enumerate(awords)}
    id2wA = {i: w for i, w in enumerate(awords)}
    w2idB = {w: i for i, w in enumerate(bwords)}
    id2wB = {i: w for i, w in enumerate(bwords)}

    # build the base matrices
    a_mtx = np.vstack([model_a.wv[w] for w in awords])
    b_mtx = np.vstack([model_b.wv[w] for w in bwords])

    # get the anchors
    a_anchor = np.vstack([a_mtx[w2idA[w], :] for w in anchorlist])
    b_anchor = np.vstack([b_mtx[w2idB[w], :] for w in anchorlist])

    # compute CCA
    cca = align_cca(a_anchor, b_anchor)

    # build and return the aligner
    aligner = CCAAligner('cca', model_a, model_b, w2idA, id2wB, a_mtx, b_mtx, anchorlist)
    aligner.set_params(cca)
    return aligner


def get_svd_aligner(model_a, model_b, anchorlist):
    # get wordmaps
    awords = list(sorted(list(model_a.wv.vocab)))
    bwords = list(sorted(list(model_b.wv.vocab)))
    w2idA = {w: i for i, w in enumerate(awords)}
    w2idB = {w: i for i, w in enumerate(bwords)}
    id2wB = {i: w for i, w in enumerate(bwords)}

    # build the base matrices
    a_mtx = np.vstack([model_a.wv[w] for w in awords])
    b_mtx = np.vstack([model_b.wv[w] for w in bwords])
    print(a_mtx.shape, b_mtx.shape)

    # get the anchors
    a_anchor = np.vstack([a_mtx[w2idA[w], :] for w in anchorlist])
    b_anchor = np.vstack([b_mtx[w2idB[w], :] for w in anchorlist])

    # get the translation matrix
    T = align_svd(a_anchor, b_anchor)

    # build and return the aligner
    aligner = SVDAligner('svd', model_a, model_b, w2idA, id2wB, a_mtx, b_mtx, anchorlist)
    aligner.set_params(T)
    return aligner

In [ ]:
aligner_cnn = get_cca_aligner(model_cnn, model_general, stopWords)
pickle.dump(aligner_cnn, open(os.path.join('./models', 'align_cnn_add.pkl'), 'wb'))
aligner_nypost = get_cca_aligner(model_nypost, model_general, stopWords)
pickle.dump(aligner_nypost, open(os.path.join('./models', 'align_nypost_add.pkl'), 'wb'))

In [ ]:
forward_cnn = pickle.load(open(os.path.join('./models', 'align_cnn_add.pkl'), 'rb'))
forward_nypost = pickle.load(open(os.path.join('./models', 'align_nypost_add.pkl'), 'rb'))
check = 'obama'
print(forward_cnn.translate_word(check, k=10), forward_nypost.translate_word(check, k=10))
ny_map=forward_nypost.translate_mtx(model_nypost.wv[check])

In [ ]:
import os
import pickle
from abc import ABC
from collections import Counter

import gensim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.cluster import KMeans
from sklearn.cross_decomposition import CCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def JS_divergence(p, q):
    M = (p + q) / 2
    return 0.5 * scipy.stats.entropy(p, M, base=2) + 0.5 * scipy.stats.entropy(q, M, base=2)

def research_topic(keywords):
    cnn_path = f"CNN_{keywords}"
    for f_name in os.listdir("./data/CNN"):
        if f_name.startswith(cnn_path):
            break
    cnn_file = open(os.path.join("./data/CNN", f_name), 'r')

    nypost_path = f"nypost_{keywords}"
    for f_name in os.listdir("./data/nypost"):
        if f_name.startswith(nypost_path):
            break
    nypost_file = open(os.path.join("./data/nypost", f_name), 'r')

    word_intersec = []
    t_vec_cnn = np.zeros(300)
    df = pd.read_csv(cnn_file)
    all_cnn = 0
    for idx, row in df.iterrows():
        for st in sent_tokenize(str(row['text'])):
            sts_list = gensim.utils.simple_preprocess(st)
            hist = Counter(sts_list)
            for k in hist:
                num = t_align.get(k, -1)
                if num >= 0:
                    word_intersec.append(k)
                    all_cnn += hist[k]
                    t_vec_cnn[num] += hist[k]
    t_vec_cnn /= all_cnn

    t_vec_nypost = np.zeros(300)
    df = pd.read_csv(nypost_file)
    all_nypost = 0
    for idx, row in df.iterrows():
        for st in sent_tokenize(str(row['text'])):
            sts_list = gensim.utils.simple_preprocess(st)
            hist = Counter(sts_list)
            for k in hist:
                num = t_align.get(k, -1)
                if num >= 0:
                    word_intersec.append(k)
                    all_nypost += hist[k]
                    t_vec_nypost[num] += hist[k]
    t_vec_nypost /= all_nypost

    topic_js = JS_divergence(t_vec_nypost, t_vec_cnn)
    topic_cos = cosine_similarity(np.array(t_vec_nypost).reshape([1, -1]), np.array(t_vec_cnn).reshape([1, -1]))

    forward_cnn = pickle.load(open(os.path.join('./models', 'align_cnn_add.pkl'), 'rb'))
    forward_nypost = pickle.load(open(os.path.join('./models', 'align_nypost_add.pkl'), 'rb'))

    dis = []
    for wd in word_intersec:
        if wd in model_general.wv.key_to_index and wd in model_cnn.wv.key_to_index:
            vec_nypost = forward_nypost.translate_mtx(model_nypost.wv[wd])
            vec_cnn = forward_cnn.translate_mtx(model_cnn.wv[wd])
            cos_sim = cosine_similarity(np.array(vec_nypost).reshape([1, -1]), np.array(vec_cnn).reshape([1, -1]))
            dis.append(cos_sim)
    mean_c = np.mean(np.array(dis))

    return topic_js, topic_cos, mean_c, t_vec_cnn, t_vec_nypost

In [ ]:
topics = ["attack", "Biden", "black", "China", "conflict", "crime", "democratic", "fair", "gun", "immigration",
          "invasion", "LGBT", "police", "protest", "refused", "republic", "Russia", "terror", "Trump", "UK", "ukraine",
          "US", "violence", "war"]

t_js_list = []
t_cos_list = []
mean_list = []
t_cnn = []
t_nypost = []

for t in topics:
    print(t)
    a, b, c, d, e = research_topic(t)
    t_js_list.append(a)
    t_cos_list.append(b)
    mean_list.append(c)
    t_cnn.append(d)
    t_nypost.append(e)

In [ ]:
plt.ylabel('content similarity by cosine similarity')
pd.DataFrame({
    'mean_simi': mean_list,
    'JS_div': t_js_list,
    'Cos_div': t_cos_list,
    't_cnn': t_cnn,
    't_nypost': t_nypost
},
    index=topics
).to_csv('./records_add.csv')